# Model Training

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import osmnx as ox
import geopandas as gpd
import networkx as nx
import folium
from folium.plugins import HeatMap
from geopy.distance import geodesic
from IPython.display import display, HTML
from geopy.geocoders import Nominatim
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import matplotlib.cm as cm 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Load Dataset

In [4]:
df=pd.read_csv('data/final_dataset.csv')

In [5]:
print(df.head()) 

  license_plate serviceOnDuty  \
0     KA51G5226            NO   
1      KA40G471            NO   
2     KA51G5240           YES   
3     KA51G5028            NO   
4     KA51G5253           YES   

                                                  id  \
0  datakaveri.org/fb0924bef803e220e0d0bc8ceaf0a1f...   
1  datakaveri.org/fb0924bef803e220e0d0bc8ceaf0a1f...   
2  datakaveri.org/fb0924bef803e220e0d0bc8ceaf0a1f...   
3  datakaveri.org/fb0924bef803e220e0d0bc8ceaf0a1f...   
4  datakaveri.org/fb0924bef803e220e0d0bc8ceaf0a1f...   

         observationDateTime location.type    location.coordinates  \
0  2022-01-31 18:30:01+00:00         Point  [77.668716, 13.133616]   
1  2022-01-31 18:30:04+00:00         Point  [77.609283, 12.941375]   
2  2022-01-31 18:30:05+00:00         Point  [77.711296, 12.801156]   
3  2022-01-31 18:30:05+00:00         Point  [77.542618, 12.976893]   
4  2022-01-31 18:30:06+00:00         Point  [77.400681, 12.965804]   

  emergencyVehicleType  location.longitudes

## Closest Ambulance

In [15]:

# Prepare KNN model
ambulance_coords = df[["location.latitudes", "location.longitudes"]].values
knn = NearestNeighbors(n_neighbors=1, algorithm="ball_tree", metric="haversine")
knn.fit(np.radians(ambulance_coords))

# Get emergency location from user
a = input("Enter the address: ")
geolocator = Nominatim(user_agent="geoapi")
emergency_location = geolocator.geocode(f"{a}, Bangalore, Karnataka")

# Check if location was found
if emergency_location:
    print(f"Emergency Location: {emergency_location.latitude}, {emergency_location.longitude}")
    latitude_degrees = emergency_location.latitude
    longitude_degrees = emergency_location.longitude

    # Convert emergency location to radians
    emergency_coords = np.radians([[latitude_degrees, longitude_degrees]])

    # Predict closest ambulance
    distances, indices = knn.kneighbors(emergency_coords)
    closest_ambulance_index = indices[0][0]
    closest_ambulance = df.iloc[closest_ambulance_index]

    # Print closest ambulance details
    print(
        f"Closest Ambulance: ID {closest_ambulance['license_plate']}, " \
        f"Distance: {distances[0][0] * 6371:.2f} km \n" \
        f"Exact Location of Ambulance: Latitude {closest_ambulance['location.latitudes']}, " \
        f"Longitude {closest_ambulance['location.longitudes']}"
    )
else:
    print("Location not found. Please try again.")


Enter the address:  BMSIT College


Emergency Location: 13.133697699999999, 77.56815456171148
Closest Ambulance: ID KA51G5230, Distance: 0.16 km 
Exact Location of Ambulance: Latitude 13.134317, Longitude 77.569458


## Estimated Time of Arrival

In [17]:
import osmnx as ox
import networkx as nx
import folium
import datetime

# Congestion levels for each hour of the day
hourly_congestion_levels = {
    0: 6, 1: 6.6, 2: 6.25, 3: 6.4, 4: 6.6, 5: 6.2, 6: 6.5, 7: 8.25, 8: 9.8, 9: 10.75, 10: 9.75,
    11: 9, 12: 8.75, 13: 9, 14: 8.8, 15: 9.2, 16: 10, 17: 11, 18: 11.75, 19: 10.3, 20: 8.5,
    21: 7.25, 22: 7, 23: 6.5
}

# Function to calculate average speed based on congestion level
def calculate_speed(congestion_level):
    base_speed_kph = 80 # Default speed on uncongested roads in km/h
    # Adjust speed inversely to congestion
    adjusted_speed_kph = base_speed_kph * (12 / congestion_level) 
    return max(5, adjusted_speed_kph)  # Ensure speed doesn't drop below 5 km/h

# Function to get current congestion level
def get_congestion_level():
    current_hour = datetime.datetime.now().hour
    return hourly_congestion_levels[current_hour]

# Load the road network
location = "Bengaluru, Karnataka, India"
graph = ox.graph_from_place(location, network_type="drive")

# Add edge lengths to the graph
graph = ox.distance.add_edge_lengths(graph)

# Define origin and destination points
orig_point = (13.133989, 77.568097)  # Example user's point (latitude, longitude)
dest_point = (12.9782998, 77.5723754)  # Example destination (latitude, longitude)

# Find nearest nodes on the graph
orig_node = ox.nearest_nodes(graph, orig_point[1], orig_point[0])  # OSM expects (lon, lat)
dest_node = ox.nearest_nodes(graph, dest_point[1], dest_point[0])

# Find the shortest path using Dijkstra
route = nx.shortest_path(graph, orig_node, dest_node, weight="length")

# Extract the route as a list of coordinates
route_coords = [(graph.nodes[node]["y"], graph.nodes[node]["x"]) for node in route]

# Load traffic lights
traffic_lights = ox.graph_to_gdfs(graph, nodes=True, edges=False)
traffic_lights = traffic_lights[traffic_lights["highway"] == "traffic_signals"]

# Find traffic signals on the route
route_traffic_signals = [
    (node, data)
    for node, data in traffic_lights.iterrows()
    if node in route
]

# Calculate travel time
congestion_level = get_congestion_level()  # Get congestion level for the current time
average_speed_kph = calculate_speed(congestion_level)  # Adjust speed based on congestion level
average_speed_mps = average_speed_kph * 1000 / 3600  # Convert to meters per second

# Calculate travel time for each segment
route_length = 0  # Total route length in meters
for u, v in zip(route[:-1], route[1:]):
    edge_data = graph[u][v][0]  # Assuming single edge between nodes
    route_length += edge_data["length"]

# Time = Distance / Speed
travel_time_seconds = route_length / average_speed_mps

# Add traffic signal delays
# Average delay per signal based on congestion level (seconds per 10 km)
average_signal_delay = 28 * 60 / 10  
total_signal_delay = len(route_traffic_signals) * average_signal_delay

# Calculate total ETA
total_eta_seconds = travel_time_seconds + total_signal_delay
total_eta_minutes = total_eta_seconds / 60

# Display results
print(f"Current Hour: {datetime.datetime.now().hour}")
print(f"Congestion Level: {congestion_level}/12")
print(f"Adjusted Speed: {average_speed_kph:.2f} km/h")
print(f"Route Length: {route_length:.2f} meters")
print(f"Travel Time (without signals): {travel_time_seconds / 60:.2f} minutes")
print(f"Number of Traffic Signals: {len(route_traffic_signals)}")
print(f"Total Signal Delay: {total_signal_delay / 60:.2f} minutes")
print(f"Estimated Time of Arrival (ETA): {total_eta_minutes:.2f} minutes")

# Create a map to visualize the route
route_map = folium.Map(location=orig_point, zoom_start=12)

# Add the route to the map
folium.PolyLine(route_coords, color="blue", weight=5, opacity=0.8).add_to(route_map)

# Define the path to your custom icon image
icon_path = "signal.png"  # Ensure the path is correct and the image is accessible

# Add markers for traffic signals on the route with a custom icon
for node, data in route_traffic_signals:
    # Create a custom icon
    my_icon = folium.CustomIcon(icon_path, icon_size=(30, 30))  # Adjust size as needed

    # Add the marker with the custom icon
    folium.Marker(
        location=(data.geometry.y, data.geometry.x),
        popup="Traffic Signal",
        icon=my_icon
    ).add_to(route_map)


# Add markers for origin and destination
folium.Marker(orig_point, popup="Origin", icon=folium.Icon(color="green")).add_to(route_map)
folium.Marker(dest_point, popup="Destination", icon=folium.Icon(color="red")).add_to(route_map)

# Display the map
route_map


Current Hour: 22
Congestion Level: 7/12
Adjusted Speed: 137.14 km/h
Route Length: 19603.82 meters
Travel Time (without signals): 8.58 minutes
Number of Traffic Signals: 3
Total Signal Delay: 8.40 minutes
Estimated Time of Arrival (ETA): 16.98 minutes


FileNotFoundError: [Errno 2] No such file or directory: 'signal.png'

# FINAL EXECUTION

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import osmnx as ox
import geopandas as gpd
import networkx as nx
import folium
from folium.plugins import HeatMap
from geopy.distance import geodesic
from IPython.display import display, HTML
from geopy.geocoders import Nominatim
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import matplotlib.cm as cm 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import datetime

# Load ambulance data (Replace with your actual data file path)
# Ensure your CSV has columns: 'license_plate', 'latitude01', 'longitude01'
df = pd.read_csv("data/final_dataset.csv")  

# Prepare KNN model for ambulance locations
ambulance_coords = df[["location.latitudes", "location.longitudes"]].values
knn = NearestNeighbors(n_neighbors=1, algorithm="ball_tree", metric="haversine")
knn.fit(np.radians(ambulance_coords))

# Congestion levels for each hour of the day
hourly_congestion_levels = {
    0: 6, 1: 6.6, 2: 6.25, 3: 6.4, 4: 6.6, 5: 6.2, 6: 6.5, 7: 8.25, 8: 9.8, 9: 10.75, 10: 9.75,
    11: 9, 12: 8.75, 13: 9, 14: 8.8, 15: 9.2, 16: 10, 17: 11, 18: 11.75, 19: 10.3, 20: 8.5,
    21: 7.25, 22: 7, 23: 6.5
}

# Function to calculate average speed based on congestion level
def calculate_speed(congestion_level):
    base_speed_kph = 120  # Default speed on uncongested roads in km/h
    adjusted_speed_kph = base_speed_kph * (congestion_level / 12)
    return max(5, adjusted_speed_kph)  # Ensure speed doesn't drop below 5 km/h

# Function to get current congestion level
def get_congestion_level():
    current_hour = datetime.datetime.now().hour
    return hourly_congestion_levels[current_hour]

# Get emergency location from user
a = input("Enter the emergency address: ")
geolocator = Nominatim(user_agent="geoapi")
emergency_location = geolocator.geocode(f"{a}, Bangalore, Karnataka")

if emergency_location:
    print(f"Emergency Location: {emergency_location.latitude}, {emergency_location.longitude}")
    latitude_degrees = emergency_location.latitude
    longitude_degrees = emergency_location.longitude

    # Convert emergency location to radians
    emergency_coords = np.radians([[latitude_degrees, longitude_degrees]])

    # Predict closest ambulance
    distances, indices = knn.kneighbors(emergency_coords)
    closest_ambulance_index = indices[0][0]
    closest_ambulance = df.iloc[closest_ambulance_index]

    # Print closest ambulance details
    print(
        f"Closest Ambulance: ID {closest_ambulance['license_plate']}, "
        f"Distance: {distances[0][0] * 6371:.2f} km \n"
        f"Exact Location of Ambulance: Latitude {closest_ambulance['location.latitudes']}, "
        f"Longitude {closest_ambulance['location.longitudes']}"
    )

    # Define origin (ambulance location) and destination (emergency location)
    orig_point = (closest_ambulance['location.latitudes'], closest_ambulance['location.longitudes'])
    dest_point = (latitude_degrees, longitude_degrees)

    # Load the road network
    location = "Bengaluru, Karnataka, India"
    graph = ox.graph_from_place(location, network_type="drive")

    # Add edge lengths to the graph
    graph = ox.distance.add_edge_lengths(graph)

    # Find nearest nodes on the graph
    orig_node = ox.nearest_nodes(graph, orig_point[1], orig_point[0])  # OSM expects (lon, lat)
    dest_node = ox.nearest_nodes(graph, dest_point[1], dest_point[0])

    # Find the shortest path using Dijkstra
    route = nx.shortest_path(graph, orig_node, dest_node, weight="length")

    # Extract the route as a list of coordinates
    route_coords = [(graph.nodes[node]["y"], graph.nodes[node]["x"]) for node in route]

    # Load traffic lights
    traffic_lights = ox.graph_to_gdfs(graph, nodes=True, edges=False)
    traffic_lights = traffic_lights[traffic_lights["highway"] == "traffic_signals"]

    # Find traffic signals on the route
    route_traffic_signals = [
        (node, data)
        for node, data in traffic_lights.iterrows()
        if node in route
    ]

    # Calculate travel time
    congestion_level = get_congestion_level()  # Get congestion level for the current time
    average_speed_kph = calculate_speed(congestion_level)  # Adjust speed based on congestion level
    average_speed_mps = average_speed_kph * 1000 / 3600  # Convert to meters per second

    # Calculate travel time for each segment
    route_length = 0  # Total route length in meters
    for u, v in zip(route[:-1], route[1:]):
        edge_data = graph[u][v][0]  # Assuming single edge between nodes
        route_length += edge_data["length"]

    # Time = Distance / Speed
    travel_time_seconds = route_length / average_speed_mps

    # Add traffic signal delays
    average_signal_delay = 28 * 60 / 10  # Average delay per signal (seconds per 10 km)
    total_signal_delay = len(route_traffic_signals) * average_signal_delay

    # Calculate total ETA
    total_eta_seconds = travel_time_seconds + total_signal_delay
    total_eta_minutes = total_eta_seconds / 60

    # Display results
    print(f"Current Hour: {datetime.datetime.now().hour}")
    print(f"Congestion Level: {congestion_level}/12")
    print(f"Adjusted Speed: {average_speed_kph:.2f} km/h")
    print(f"Route Length: {route_length:.2f} meters")
    print(f"Travel Time (without signals): {travel_time_seconds / 60:.2f} minutes")
    print(f"Number of Traffic Signals: {len(route_traffic_signals)}")
    print(f"Total Signal Delay: {total_signal_delay / 60:.2f} minutes")
    print(f"Estimated Time of Arrival (ETA): {total_eta_minutes:.2f} minutes")

    # Create a map to visualize the route
    route_map = folium.Map(location=orig_point, zoom_start=12)

    # Add the route to the map
    folium.PolyLine(route_coords, color="blue", weight=5, opacity=0.8).add_to(route_map)

    # Define the path to your custom icon image
    icon_path = "signal.jpg"  # Ensure the path is correct and the image is accessible

    # Add markers for traffic signals on the route
    for node, data in route_traffic_signals:
        folium.Marker(
            location=(data.geometry.y, data.geometry.x),
            popup="Traffic Signal",
            icon=folium.Icon(color="darkpurple", icon="exclamation-sign"),
        ).add_to(route_map)

    # Add markers for origin and destination
    folium.Marker(orig_point, popup="Ambulance", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(dest_point, popup="Emergency", icon=folium.Icon(color="red")).add_to(route_map)

    # Display the map
    display(route_map)
else:
    print("Location not found. Please try again.")


Enter the emergency address:  orion mall


Emergency Location: 13.0109788, 77.55522384998105
Closest Ambulance: ID KA02G1622, Distance: 0.87 km 
Exact Location of Ambulance: Latitude 13.017893, Longitude 77.559021
Current Hour: 14
Congestion Level: 8.8/12
Adjusted Speed: 88.00 km/h
Route Length: 2463.12 meters
Travel Time (without signals): 1.68 minutes
Number of Traffic Signals: 2
Total Signal Delay: 5.60 minutes
Estimated Time of Arrival (ETA): 7.28 minutes
